# Interactive Experiment Runner

This notebook allows you to run reflection experiments with configurable parameters.

**Configurable Parameters:**
- Number of runs
- Reflection levels (0-5)
- Thinking conditions (ON/OFF)
- Dataset selection (ETHICS, MoralChoice, or both)
- Sample size
- Model settings

## 1. Setup & Imports

In [ ]:
import sys
import os
import re
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path(os.getcwd()).parent
sys.path.insert(0, str(PROJECT_ROOT))
os.chdir(PROJECT_ROOT)

# Load environment variables
if os.path.exists('.env'):
    with open('.env', 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#'):
                match = re.match(r'(\w+)\s*=\s*["\']?([^"\']+)["\']?', line)
                if match:
                    os.environ[match.group(1)] = match.group(2)

print(f"Project root: {PROJECT_ROOT}")

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import seaborn as sns

# Project imports
import config
from prompts import get_ethics_prompt, get_moralchoice_prompt
from src.api import call_with_rate_limit, call_claude, APIResponse
from src.extraction import (
    extract_ethics_answer,
    extract_moralchoice_with_confidence,
    count_reasoning_markers,
    count_uncertainty_markers
)

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print(f"Model: {config.MODEL}")
print(f"API Key loaded: {'Yes' if config.ANTHROPIC_API_KEY else 'No'}")

## 2. Experiment Configuration

**Modify these parameters to customize your experiment:**

In [ ]:
# ============================================================================
# EXPERIMENT CONFIGURATION - MODIFY THESE PARAMETERS
# ============================================================================

# Reflection levels to test (0-5)
# Level 0: Direct answer only
# Level 1: Minimal reasoning
# Level 2: Chain-of-thought
# Level 3: Structured reasoning
# Level 4: Adversarial (consider counterarguments)
# Level 5: Two-pass reflection
LEVELS = [0, 2, 4]  # Example: test direct, CoT, and adversarial

# Thinking conditions
THINKING_CONDITIONS = [False, True]  # [OFF, ON] or just [False] or [True]

# Number of runs per condition (for consistency measurement)
N_RUNS = 1  # Set to 3 for full consistency analysis

# Dataset selection
RUN_ETHICS = True
RUN_MORALCHOICE = True

# Sample size (set to None for full dataset)
SAMPLE_SIZE = 10  # Start small for testing

# Include confidence in MoralChoice prompts
INCLUDE_CONFIDENCE = True

# Rate limiting (calls per minute)
RATE_LIMIT = 50

# Output settings
SAVE_CHECKPOINTS = True
CHECKPOINT_DIR = "results/notebook"

# ============================================================================

# Calculate estimated API calls
def estimate_calls():
    ethics_items = SAMPLE_SIZE if SAMPLE_SIZE else 1500
    mc_items = SAMPLE_SIZE if SAMPLE_SIZE else 500
    
    # Level 5 requires 2 calls per item
    calls_per_item = sum(2 if l == 5 else 1 for l in LEVELS)
    calls_per_condition = calls_per_item * len(THINKING_CONDITIONS) * N_RUNS
    
    ethics_calls = ethics_items * calls_per_condition if RUN_ETHICS else 0
    mc_calls = mc_items * calls_per_condition if RUN_MORALCHOICE else 0
    
    return ethics_calls, mc_calls

ethics_calls, mc_calls = estimate_calls()
total_calls = ethics_calls + mc_calls

print("=" * 50)
print("EXPERIMENT CONFIGURATION")
print("=" * 50)
print(f"Levels: {LEVELS}")
print(f"Thinking conditions: {['OFF' if not t else 'ON' for t in THINKING_CONDITIONS]}")
print(f"Runs per condition: {N_RUNS}")
print(f"Sample size: {SAMPLE_SIZE or 'Full dataset'}")
print(f"\nDatasets:")
print(f"  ETHICS: {'Yes' if RUN_ETHICS else 'No'}")
print(f"  MoralChoice: {'Yes' if RUN_MORALCHOICE else 'No'}")
print(f"\nEstimated API calls: ~{total_calls}")
print(f"Estimated time at {RATE_LIMIT} calls/min: ~{total_calls / RATE_LIMIT:.1f} minutes")

## 3. Load Data

In [ ]:
def load_ethics_data(sample_size=None):
    """Load ETHICS dataset with optional balanced sampling."""
    ethics = pd.read_csv("data/ethics_sample.csv")
    
    if sample_size:
        per_subscale = sample_size // 3
        subscales = []
        for subscale in ['commonsense', 'deontology', 'virtue']:
            subset = ethics[ethics['subscale'] == subscale].head(per_subscale)
            subscales.append(subset)
        ethics = pd.concat(subscales, ignore_index=True)
    
    return ethics

def load_moralchoice_data(sample_size=None):
    """Load MoralChoice dataset with optional balanced sampling."""
    mc = pd.read_csv("data/moralchoice_sample.csv")
    
    if sample_size:
        per_level = sample_size // 2
        low_amb = mc[mc['ambiguity'] == 'low'].head(per_level)
        high_amb = mc[mc['ambiguity'] == 'high'].head(per_level)
        mc = pd.concat([low_amb, high_amb], ignore_index=True)
    
    return mc

# Load datasets
if RUN_ETHICS:
    ethics_data = load_ethics_data(SAMPLE_SIZE)
    print(f"ETHICS loaded: {len(ethics_data)} items")
    print(f"  Subscales: {ethics_data['subscale'].value_counts().to_dict()}")

if RUN_MORALCHOICE:
    mc_data = load_moralchoice_data(SAMPLE_SIZE)
    print(f"\nMoralChoice loaded: {len(mc_data)} items")
    if 'ambiguity' in mc_data.columns:
        print(f"  Ambiguity: {mc_data['ambiguity'].value_counts().to_dict()}")

## 4. Preview Data & Prompts

In [ ]:
# Preview sample items
if RUN_ETHICS:
    print("=" * 50)
    print("ETHICS SAMPLE")
    print("=" * 50)
    sample_ethics = ethics_data.iloc[0]
    print(f"Scenario: {sample_ethics['scenario'][:200]}...")
    print(f"Label: {sample_ethics['label']}")
    print(f"Subscale: {sample_ethics['subscale']}")

if RUN_MORALCHOICE:
    print("\n" + "=" * 50)
    print("MORALCHOICE SAMPLE")
    print("=" * 50)
    sample_mc = mc_data.iloc[0]
    print(f"Context: {sample_mc.get('context', 'N/A')[:100]}...")
    print(f"Option A: {sample_mc['option_a'][:100]}...")
    print(f"Option B: {sample_mc['option_b'][:100]}...")

In [ ]:
# Preview prompts at different levels
def preview_prompts(dataset='ethics', levels=[0, 2, 5]):
    """Preview how prompts look at different levels."""
    print(f"\n{'='*60}")
    print(f"PROMPT PREVIEW - {dataset.upper()}")
    print(f"{'='*60}")
    
    if dataset == 'ethics' and RUN_ETHICS:
        sample = ethics_data.iloc[0]
        for level in levels:
            print(f"\n--- Level {level} ---")
            prompt = get_ethics_prompt(level, sample['scenario'])
            print(prompt[:500] + "..." if len(prompt) > 500 else prompt)
    
    elif dataset == 'moralchoice' and RUN_MORALCHOICE:
        sample = mc_data.iloc[0]
        for level in levels:
            print(f"\n--- Level {level} ---")
            prompt = get_moralchoice_prompt(
                level, 
                sample.get('context', ''),
                sample['option_a'],
                sample['option_b'],
                include_confidence=INCLUDE_CONFIDENCE
            )
            print(prompt[:500] + "..." if len(prompt) > 500 else prompt)

# Preview prompts for configured levels
preview_prompts('ethics', LEVELS[:3])  # Show first 3 levels

## 5. Core Experiment Functions

In [ ]:
def run_single_ethics(row, level, thinking, rate_limit=True):
    """
    Run single ETHICS item at given condition.
    
    Args:
        row: DataFrame row with scenario, label
        level: Reflection level (0-5)
        thinking: Whether to enable extended thinking
        rate_limit: Whether to apply rate limiting
    
    Returns:
        Dict with response data and metrics
    """
    api_call = call_with_rate_limit if rate_limit else call_claude
    
    if level == 5:
        # Two-pass reflection
        prompt1 = get_ethics_prompt(5, row['scenario'])
        response1 = api_call(prompt1, thinking)
        
        prompt2 = get_ethics_prompt(5, row['scenario'], response1.content)
        response2 = api_call(prompt2, thinking)
        
        content = response2.content
        full_response = f"[PASS1]\n{response1.content}\n\n[PASS2]\n{response2.content}"
        thinking_content = response2.thinking
        input_tokens = response1.input_tokens + response2.input_tokens
        output_tokens = response1.output_tokens + response2.output_tokens
    else:
        prompt = get_ethics_prompt(level, row['scenario'])
        response = api_call(prompt, thinking)
        
        content = response.content
        full_response = response.content
        thinking_content = response.thinking
        input_tokens = response.input_tokens
        output_tokens = response.output_tokens
    
    # Extract answer
    extracted = extract_ethics_answer(content)
    
    return {
        'item_id': row['item_id'],
        'subscale': row['subscale'],
        'scenario': row['scenario'][:200],
        'correct_answer': row['label'],
        'level': level,
        'thinking': thinking,
        'response': full_response,
        'thinking_content': thinking_content,
        'extracted_answer': extracted,
        'correct': extracted == row['label'] if extracted else None,
        'response_length': len(full_response.split()),
        'reasoning_markers': count_reasoning_markers(full_response),
        'uncertainty_markers': count_uncertainty_markers(full_response),
        'input_tokens': input_tokens,
        'output_tokens': output_tokens,
        'timestamp': datetime.now().isoformat(),
    }


def run_single_moralchoice(row, level, thinking, include_confidence=True, rate_limit=True):
    """
    Run single MoralChoice item at given condition.
    
    Args:
        row: DataFrame row with context, option_a, option_b
        level: Reflection level (0-5)
        thinking: Whether to enable extended thinking
        include_confidence: Whether to ask for confidence score
        rate_limit: Whether to apply rate limiting
    
    Returns:
        Dict with response data and metrics
    """
    api_call = call_with_rate_limit if rate_limit else call_claude
    context = row.get('context', '')
    
    if level == 5:
        prompt1 = get_moralchoice_prompt(5, context, row['option_a'], row['option_b'],
                                         include_confidence=include_confidence)
        response1 = api_call(prompt1, thinking)
        
        prompt2 = get_moralchoice_prompt(5, context, row['option_a'], row['option_b'],
                                         previous_response=response1.content,
                                         include_confidence=include_confidence)
        response2 = api_call(prompt2, thinking)
        
        content = response2.content
        full_response = f"[PASS1]\n{response1.content}\n\n[PASS2]\n{response2.content}"
        thinking_content = response2.thinking
        input_tokens = response1.input_tokens + response2.input_tokens
        output_tokens = response1.output_tokens + response2.output_tokens
    else:
        prompt = get_moralchoice_prompt(level, context, row['option_a'], row['option_b'],
                                        include_confidence=include_confidence)
        response = api_call(prompt, thinking)
        
        content = response.content
        full_response = response.content
        thinking_content = response.thinking
        input_tokens = response.input_tokens
        output_tokens = response.output_tokens
    
    # Extract answer and confidence
    extraction = extract_moralchoice_with_confidence(content)
    
    return {
        'item_id': row['item_id'],
        'context': row.get('context', '')[:100],
        'option_a': row['option_a'][:100],
        'option_b': row['option_b'][:100],
        'ambiguity': row.get('ambiguity', None),
        'level': level,
        'thinking': thinking,
        'response': full_response,
        'thinking_content': thinking_content,
        'extracted_answer': extraction['answer'],
        'confidence': extraction['confidence'],
        'confidence_category': extraction['confidence_category'],
        'response_length': len(full_response.split()),
        'reasoning_markers': count_reasoning_markers(full_response),
        'uncertainty_markers': count_uncertainty_markers(full_response),
        'input_tokens': input_tokens,
        'output_tokens': output_tokens,
        'timestamp': datetime.now().isoformat(),
    }

print("Core functions loaded.")

## 6. Run Experiment

In [ ]:
def run_experiment(data, dataset_type, levels, thinking_conditions, n_runs,
                   include_confidence=True, save_checkpoints=True, checkpoint_dir="results/notebook"):
    """
    Run experiment on dataset with specified parameters.
    
    Args:
        data: DataFrame with experiment items
        dataset_type: 'ethics' or 'moralchoice'
        levels: List of reflection levels to test
        thinking_conditions: List of thinking conditions [False, True]
        n_runs: Number of runs per condition
        include_confidence: For MoralChoice, whether to ask for confidence
        save_checkpoints: Whether to save intermediate results
        checkpoint_dir: Directory for checkpoint files
    
    Returns:
        DataFrame with all results
    """
    results = []
    
    # Create checkpoint directory
    if save_checkpoints:
        Path(checkpoint_dir).mkdir(parents=True, exist_ok=True)
    
    total_conditions = len(levels) * len(thinking_conditions) * n_runs
    condition_num = 0
    
    for run in range(n_runs):
        for thinking in thinking_conditions:
            for level in levels:
                condition_num += 1
                thinking_label = "ON" if thinking else "OFF"
                
                print(f"\n[{condition_num}/{total_conditions}] "
                      f"{dataset_type.upper()} Run {run+1}, Level {level}, Thinking {thinking_label}")
                
                for _, row in tqdm(data.iterrows(), total=len(data),
                                   desc=f"R{run+1}-L{level}-{thinking_label}"):
                    try:
                        if dataset_type == 'ethics':
                            result = run_single_ethics(row, level, thinking)
                        else:
                            result = run_single_moralchoice(row, level, thinking, include_confidence)
                        
                        result['run'] = run
                        results.append(result)
                        
                    except Exception as e:
                        print(f"Error on item {row['item_id']}: {e}")
                        results.append({
                            'item_id': row['item_id'],
                            'level': level,
                            'thinking': thinking,
                            'run': run,
                            'error': str(e),
                            'timestamp': datetime.now().isoformat(),
                        })
                
                # Save checkpoint
                if save_checkpoints:
                    checkpoint_df = pd.DataFrame(results)
                    checkpoint_df.to_csv(f"{checkpoint_dir}/{dataset_type}_checkpoint.csv", index=False)
    
    return pd.DataFrame(results)

print("Experiment runner loaded.")

In [ ]:
# Run the experiment
start_time = datetime.now()
print("=" * 60)
print("STARTING EXPERIMENT")
print("=" * 60)
print(f"Start time: {start_time}")

ethics_results = None
mc_results = None

if RUN_ETHICS:
    print("\n" + "=" * 40)
    print("RUNNING ETHICS EXPERIMENT")
    print("=" * 40)
    ethics_results = run_experiment(
        ethics_data, 'ethics',
        levels=LEVELS,
        thinking_conditions=THINKING_CONDITIONS,
        n_runs=N_RUNS,
        save_checkpoints=SAVE_CHECKPOINTS,
        checkpoint_dir=CHECKPOINT_DIR
    )
    print(f"\nETHICS complete: {len(ethics_results)} observations")

if RUN_MORALCHOICE:
    print("\n" + "=" * 40)
    print("RUNNING MORALCHOICE EXPERIMENT")
    print("=" * 40)
    mc_results = run_experiment(
        mc_data, 'moralchoice',
        levels=LEVELS,
        thinking_conditions=THINKING_CONDITIONS,
        n_runs=N_RUNS,
        include_confidence=INCLUDE_CONFIDENCE,
        save_checkpoints=SAVE_CHECKPOINTS,
        checkpoint_dir=CHECKPOINT_DIR
    )
    print(f"\nMoralChoice complete: {len(mc_results)} observations")

end_time = datetime.now()
print("\n" + "=" * 60)
print("EXPERIMENT COMPLETE")
print("=" * 60)
print(f"Duration: {end_time - start_time}")

## 7. Quick Analysis & Visualization

In [ ]:
def quick_analysis(results, dataset_type='ethics'):
    """Generate quick summary statistics."""
    if results is None or len(results) == 0:
        print(f"No {dataset_type} results to analyze.")
        return
    
    print(f"\n{'='*50}")
    print(f"{dataset_type.upper()} RESULTS SUMMARY")
    print(f"{'='*50}")
    
    # Basic stats
    print(f"\nTotal observations: {len(results)}")
    print(f"Unique items: {results['item_id'].nunique()}")
    
    # Check for errors
    if 'error' in results.columns:
        errors = results['error'].notna().sum()
        print(f"Errors: {errors}")
    
    # Extraction success
    if dataset_type == 'ethics':
        extraction_col = 'extracted_answer'
    else:
        extraction_col = 'extracted_answer'
    
    if extraction_col in results.columns:
        extraction_failures = results[extraction_col].isna().sum()
        print(f"Extraction failures: {extraction_failures} ({extraction_failures/len(results)*100:.1f}%)")
    
    # Accuracy by condition (ETHICS)
    if dataset_type == 'ethics' and 'correct' in results.columns:
        print("\nAccuracy by Level & Thinking:")
        accuracy = results.groupby(['level', 'thinking'])['correct'].mean().unstack()
        accuracy.columns = ['Thinking OFF', 'Thinking ON']
        print(accuracy.round(3).to_string())
    
    # Answer distribution (MoralChoice)
    if dataset_type == 'moralchoice' and 'extracted_answer' in results.columns:
        print("\nAnswer Distribution by Level:")
        answer_dist = results.groupby('level')['extracted_answer'].value_counts(normalize=True).unstack()
        print(answer_dist.round(3).to_string())
        
        if 'confidence' in results.columns:
            print("\nMean Confidence by Level:")
            conf = results.groupby(['level', 'thinking'])['confidence'].mean().unstack()
            conf.columns = ['Thinking OFF', 'Thinking ON']
            print(conf.round(1).to_string())
    
    # Token usage
    if 'input_tokens' in results.columns:
        print("\nToken Usage by Level:")
        tokens = results.groupby('level').agg({
            'input_tokens': 'mean',
            'output_tokens': 'mean'
        }).round(0)
        print(tokens.to_string())

# Run quick analysis
if ethics_results is not None:
    quick_analysis(ethics_results, 'ethics')

if mc_results is not None:
    quick_analysis(mc_results, 'moralchoice')

In [ ]:
def plot_results(results, dataset_type='ethics'):
    """Generate visualization of results."""
    if results is None or len(results) == 0:
        print(f"No {dataset_type} results to plot.")
        return
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Main metric by level and thinking
    ax1 = axes[0]
    
    if dataset_type == 'ethics' and 'correct' in results.columns:
        # Accuracy plot
        accuracy = results.groupby(['level', 'thinking'])['correct'].mean().reset_index()
        accuracy['thinking_label'] = accuracy['thinking'].map({True: 'Thinking ON', False: 'Thinking OFF'})
        
        sns.lineplot(data=accuracy, x='level', y='correct', hue='thinking_label',
                     marker='o', markersize=10, ax=ax1)
        ax1.set_xlabel('Reflection Level')
        ax1.set_ylabel('Accuracy')
        ax1.set_title(f'{dataset_type.upper()}: Accuracy by Level')
        ax1.set_ylim(0, 1.05)
        ax1.legend(title='')
    
    elif dataset_type == 'moralchoice' and 'extracted_answer' in results.columns:
        # Preference plot (% choosing A)
        results_clean = results[results['extracted_answer'].isin(['A', 'B'])].copy()
        results_clean['chose_A'] = (results_clean['extracted_answer'] == 'A').astype(int)
        
        pref = results_clean.groupby(['level', 'thinking'])['chose_A'].mean().reset_index()
        pref['thinking_label'] = pref['thinking'].map({True: 'Thinking ON', False: 'Thinking OFF'})
        
        sns.lineplot(data=pref, x='level', y='chose_A', hue='thinking_label',
                     marker='o', markersize=10, ax=ax1)
        ax1.set_xlabel('Reflection Level')
        ax1.set_ylabel('Proportion Choosing A')
        ax1.set_title(f'{dataset_type.upper()}: Preference by Level')
        ax1.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
        ax1.set_ylim(0, 1)
        ax1.legend(title='')
    
    # Plot 2: Response length by level
    ax2 = axes[1]
    
    if 'response_length' in results.columns:
        length = results.groupby(['level', 'thinking'])['response_length'].mean().reset_index()
        length['thinking_label'] = length['thinking'].map({True: 'Thinking ON', False: 'Thinking OFF'})
        
        sns.barplot(data=length, x='level', y='response_length', hue='thinking_label', ax=ax2)
        ax2.set_xlabel('Reflection Level')
        ax2.set_ylabel('Mean Response Length (words)')
        ax2.set_title('Response Length by Level')
        ax2.legend(title='')
    
    plt.tight_layout()
    plt.show()

# Generate plots
if ethics_results is not None:
    plot_results(ethics_results, 'ethics')

if mc_results is not None:
    plot_results(mc_results, 'moralchoice')

## 8. Save Final Results

In [ ]:
# Save final results
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = Path("results/notebook")
output_dir.mkdir(parents=True, exist_ok=True)

if ethics_results is not None:
    output_path = output_dir / f"ethics_results_{timestamp}.csv"
    ethics_results.to_csv(output_path, index=False)
    print(f"ETHICS results saved to: {output_path}")

if mc_results is not None:
    output_path = output_dir / f"moralchoice_results_{timestamp}.csv"
    mc_results.to_csv(output_path, index=False)
    print(f"MoralChoice results saved to: {output_path}")

## 9. Single Item Testing

Test individual items to debug or explore responses.

In [ ]:
def test_single_item(item_idx=0, dataset='ethics', level=2, thinking=False, show_response=True):
    """
    Test a single item for debugging.
    
    Args:
        item_idx: Index of item in dataset
        dataset: 'ethics' or 'moralchoice'
        level: Reflection level
        thinking: Enable extended thinking
        show_response: Print full response
    """
    if dataset == 'ethics':
        data = load_ethics_data(None)  # Load full for selection
        row = data.iloc[item_idx]
        result = run_single_ethics(row, level, thinking, rate_limit=False)
    else:
        data = load_moralchoice_data(None)
        row = data.iloc[item_idx]
        result = run_single_moralchoice(row, level, thinking, INCLUDE_CONFIDENCE, rate_limit=False)
    
    print(f"\n{'='*60}")
    print(f"SINGLE ITEM TEST - {dataset.upper()}")
    print(f"{'='*60}")
    print(f"Item ID: {result['item_id']}")
    print(f"Level: {level}, Thinking: {'ON' if thinking else 'OFF'}")
    
    if dataset == 'ethics':
        print(f"\nScenario: {row['scenario'][:200]}...")
        print(f"\nCorrect: {result['correct_answer']}")
        print(f"Extracted: {result['extracted_answer']}")
        print(f"Correct: {result['correct']}")
    else:
        print(f"\nContext: {row.get('context', 'N/A')[:100]}")
        print(f"Option A: {row['option_a'][:100]}")
        print(f"Option B: {row['option_b'][:100]}")
        print(f"\nExtracted Answer: {result['extracted_answer']}")
        print(f"Confidence: {result.get('confidence', 'N/A')}")
    
    print(f"\nTokens: {result['input_tokens']} in, {result['output_tokens']} out")
    print(f"Response length: {result['response_length']} words")
    
    if show_response:
        print(f"\n{'='*60}")
        print("FULL RESPONSE:")
        print(f"{'='*60}")
        print(result['response'])
        
        if result.get('thinking_content'):
            print(f"\n{'='*60}")
            print("THINKING CONTENT:")
            print(f"{'='*60}")
            print(result['thinking_content'][:1000] + "..." if len(result['thinking_content']) > 1000 else result['thinking_content'])
    
    return result

# Example: Test item 0 at level 2 with thinking OFF
# Uncomment to run:
# test_result = test_single_item(item_idx=0, dataset='ethics', level=2, thinking=False)

## 10. Custom Prompt Testing

Test custom prompts or modified scenarios.

In [ ]:
def test_custom_prompt(prompt, thinking=False):
    """
    Test a custom prompt directly.
    
    Args:
        prompt: Custom prompt string
        thinking: Enable extended thinking
    """
    print(f"\n{'='*60}")
    print("CUSTOM PROMPT TEST")
    print(f"{'='*60}")
    print(f"Thinking: {'ON' if thinking else 'OFF'}")
    print(f"\nPrompt:\n{prompt}")
    
    response = call_claude(prompt, thinking)
    
    print(f"\n{'='*60}")
    print("RESPONSE:")
    print(f"{'='*60}")
    print(response.content)
    
    if response.thinking:
        print(f"\n{'='*60}")
        print("THINKING:")
        print(f"{'='*60}")
        print(response.thinking[:500] + "..." if len(response.thinking) > 500 else response.thinking)
    
    print(f"\nTokens: {response.input_tokens} in, {response.output_tokens} out")
    
    return response

# Example custom prompt test:
# custom_response = test_custom_prompt(
#     """Scenario: A person finds $100 on the ground with no owner in sight.
#     
#     Is keeping the money wrong or not wrong? Think step by step.""",
#     thinking=False
# )

## 11. Load Previous Results

Load and analyze results from previous runs.

In [ ]:
def load_previous_results(path=None, dataset_type='ethics'):
    """
    Load results from a previous experiment run.
    
    Args:
        path: Path to CSV file (or None to use default)
        dataset_type: 'ethics' or 'moralchoice'
    """
    if path is None:
        path = f"results/processed/{dataset_type}_results.csv"
    
    if not os.path.exists(path):
        print(f"File not found: {path}")
        return None
    
    results = pd.read_csv(path)
    print(f"Loaded {len(results)} observations from {path}")
    print(f"Levels: {sorted(results['level'].unique())}")
    print(f"Thinking conditions: {results['thinking'].unique()}")
    
    return results

# Load main experiment results:
# prev_ethics = load_previous_results(dataset_type='ethics')
# prev_mc = load_previous_results(dataset_type='moralchoice')

---

## Quick Reference

### Reflection Levels
| Level | Name | Description |
|-------|------|-------------|
| 0 | Direct | One-word answer only |
| 1 | Minimal | Simple question |
| 2 | CoT | Step-by-step reasoning |
| 3 | Structured | Principle-based scaffolding |
| 4 | Adversarial | Consider counterarguments |
| 5 | Two-Pass | Initial + balanced reflection |

### Key Functions
- `run_experiment()` - Run full experiment with parameters
- `test_single_item()` - Test one item for debugging
- `test_custom_prompt()` - Test any custom prompt
- `quick_analysis()` - Generate summary stats
- `plot_results()` - Visualize results
- `load_previous_results()` - Load saved results